In [5]:
import pandas as pd

df = pd.read_csv("./train.csv")
df.head()


,Unnamed: 0,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os_bit,graphic_card_gb,weight,display_size,...,M1.1,Other_P,Pentium Quad,Ryzen 3,Ryzen 5,Ryzen 7,Ryzen 9,DOS,Mac,Windows
0,0,10.0,2,1,0,1,1,0,1.0,NaN,...,0,1,0,0,0,0,0,0,0,1
1,1,10.0,2,1,0,3,1,0,1.5,NaN,...,0,1,0,0,0,0,0,0,0,1
2,2,10.0,2,1,2,0,1,0,1.0,NaN,...,0,1,0,0,0,0,0,0,0,1
3,3,10.0,2,1,2,0,1,0,1.0,NaN,...,0,1,0,0,0,0,0,0,0,1
4,4,10.0,2,1,4,0,1,0,1.0,NaN,...,0,1,0,0,0,0,0,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       896 non-null    int64  
 1   processor_gnrtn  657 non-null    float64
 2   ram_gb           896 non-null    int64  
 3   ram_type         896 non-null    int64  
 4   ssd              896 non-null    int64  
 5   hdd              896 non-null    int64  
 6   os_bit           896 non-null    int64  
 7   graphic_card_gb  896 non-null    int64  
 8   weight           896 non-null    float64
 9   display_size     564 non-null    float64
 10  warranty         896 non-null    int64  
 11  Touchscreen      896 non-null    int64  
 12  msoffice         896 non-null    int64  
 13  latest_price     896 non-null    int64  
 14  old_price        896 non-null    int64  
 15  discount         896 non-null    int64  
 16  star_rating      896 non-null    float64
 17  ratings         

In [4]:
missing_cols = [col for col in df.columns if df[col].isnull().sum()>0]
print(missing_cols)

['processor_gnrtn', 'display_size']


In [3]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor


def tree_imputation(df):
    missing_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
    non_missing_cols = [col for col in df.columns if df[col].isnull().sum() == 0]
    # num_cols = [col for col in missing_cols if df[col].dtype != 'object']

    # df = df[num_cols]
    for col in missing_cols:

        # Defining a new bagging model for each attribute  
        model = BaggingRegressor(DecisionTreeRegressor(), n_estimators = 40, max_samples = 1.0, max_features = 1.0, bootstrap = False, n_jobs = -1)

        col_missing = df[df[col].isnull()]
        temp = df.drop(df[df[col].isnull()].index, axis = 0)

        # print(temp.columns)
        # X = temp.drop(col, axis = 1)
        X = temp.loc[:, non_missing_cols]
        y = temp[col]

        model.fit(X, y)

        y_pred = model.predict(col_missing[non_missing_cols])
        # col_missing[col] = y_pred

        df.loc[col_missing.index, col] = y_pred
        
    return df

In [4]:
temp = tree_imputation(df)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       896 non-null    int64  
 1   processor_gnrtn  896 non-null    float64
 2   ram_gb           896 non-null    int64  
 3   ram_type         896 non-null    int64  
 4   ssd              896 non-null    int64  
 5   hdd              896 non-null    int64  
 6   os_bit           896 non-null    int64  
 7   graphic_card_gb  896 non-null    int64  
 8   weight           896 non-null    float64
 9   display_size     896 non-null    float64
 10  warranty         896 non-null    int64  
 11  Touchscreen      896 non-null    int64  
 12  msoffice         896 non-null    int64  
 13  latest_price     896 non-null    int64  
 14  old_price        896 non-null    int64  
 15  discount         896 non-null    int64  
 16  star_rating      896 non-null    float64
 17  ratings         

In [5]:
temp

,Unnamed: 0,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os_bit,graphic_card_gb,weight,display_size,...,M1.1,Other_P,Pentium Quad,Ryzen 3,Ryzen 5,Ryzen 7,Ryzen 9,DOS,Mac,Windows
0,0,10.000,2,1,0,1,1,0,1.0,15.60,...,0,1,0,0,0,0,0,0,0,1
1,1,10.000,2,1,0,3,1,0,1.5,14.72,...,0,1,0,0,0,0,0,0,0,1
2,2,10.000,2,1,2,0,1,0,1.0,14.00,...,0,1,0,0,0,0,0,0,0,1
3,3,10.000,2,1,2,0,1,0,1.0,14.00,...,0,1,0,0,0,0,0,0,0,1
4,4,10.000,2,1,4,0,1,0,1.0,14.00,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,891,10.525,2,1,1,0,1,0,1.5,16.00,...,0,0,0,0,0,0,1,0,0,1
892,892,11.000,2,1,1,0,1,4,1.5,16.00,...,0,0,0,0,0,0,1,0,0,1
893,893,11.000,2,1,1,0,1,4,1.5,16.00,...,0,0,0,0,0,0,1,0,0,1
894,894,11.000,2,5,0,3,0,0,1.5,15.60,...,0,1,0,0,0,0,0,0,0,1
